# AfriSAR: Aboveground Biomass for Lope, Mabounie, Mondah, and Rabi Sites, Gabon

## Description
This dataset provides gridded estimates of aboveground biomass (AGB) for four sites in Gabon at 0.25 ha (50 m) resolution derived with field measurements and airborne LiDAR data collected from 2010 to 2016. The sites represent a mix of forested, savannah, and some agricultural and disturbed landcover types: Lope site, within Lope National Park; Mabounie, mostly forested site; Mondah Forest, protected area; and the Rabi forest site, part of the Smithsonian Institution of Global Earth Observatories world-wide network of forest plots. Plot-level biophysical measurements of tree diameter and tree height (or estimated by allometry) were performed at 1 ha and 0.25 ha scales on multiple plots at each site and used to derive AGB for each tree and then summed for each plot. Aerial LiDAR scans were used to construct digital elevation models (DEM) and digital surface models (DSM), and then the DEM and DSM were used to construct a canopy height model (CHM) at 1 m resolution. After checking site-plot locations against the CHM, mean canopy height (MCH) was computed over each 0.25 ha. A single regression model relating MCH and AGB estimates, incorporating local height based on the trunk DBH (HD) relationships, was produced for all sites and combined with the CHM layer to construct biomass maps at 0.25 ha resolution.

In [40]:
# !pip install pystac-client
import rioxarray
import stackstac
import rasterio as rio
from pystac_client import Client
import hvplot.xarray

import warnings
warnings.filterwarnings("ignore")

## Declare your collection of interest

You can discover available collections the follwoing ways:

1. Use the {STAC_API_URL}/collections API endpoint (JSON response)
2. Programmatically using pystac (see example in the list-collections.ipynb notebook)
3. In the STAC Browser : http://delta-staging-stac-browser.s3-website-us-east-1.amazonaws.com/


In [41]:
STAC_API_URL = 'https://stac.maap-project.org/'
collection = 'AfriSAR_AGB_Maps_1681'

In [42]:
# STAC API root URL
URL = STAC_API_URL

# custom headers
headers = []

cat = Client.open(URL, headers=headers)


## Listing all the available collections

In [35]:
for collection in cat.get_collections():
    print(collection)


<CollectionClient id=AfriSAR_UAVSAR_Coreg_SLC>
<CollectionClient id=Landsat8_SurfaceReflectance>
<CollectionClient id=AfriSAR_AGB_Maps_1681>
<CollectionClient id=ABLVIS1B>
<CollectionClient id=GEDI02_B>
<CollectionClient id=AFLVIS2>
<CollectionClient id=BIOSAR1>
<CollectionClient id=AFRISAR_DLR>
<CollectionClient id=GEDI02_A>


## Getting the collection of intrest 'AfriSAR_AGB_Maps_1681'

In [36]:
collection = cat.get_collection('AfriSAR_AGB_Maps_1681')
collection

ID: AfriSAR_AGB_Maps_1681
"Title: AfriSAR: Aboveground Biomass for Lope, Mabounie, Mondah, and Rabi Sites, Gabon"
"Description: This dataset provides gridded estimates of aboveground biomass (AGB) for four sites in Gabon at 0.25 ha (50 m) resolution derived with field measurements and airborne LiDAR data collected from 2010 to 2016. The sites represent a mix of forested, savannah, and some agricultural and disturbed landcover types: Lope site, within Lope National Park; Mabounie, mostly forested site; Mondah Forest, protected area; and the Rabi forest site, part of the Smithsonian Institution of Global Earth Observatories world-wide network of forest plots. Plot-level biophysical measurements of tree diameter and tree height (or estimated by allometry) were performed at 1 ha and 0.25 ha scales on multiple plots at each site and used to derive AGB for each tree and then summed for each plot. Aerial LiDAR scans were used to construct digital elevation models (DEM) and digital surface models (DSM), and then the DEM and DSM were used to construct a canopy height model (CHM) at 1 m resolution. After checking site-plot locations against the CHM, mean canopy height (MCH) was computed over each 0.25 ha. A single regression model relating MCH and AGB estimates, incorporating local height based on the trunk DBH (HD) relationships, was produced for all sites and combined with the CHM layer to construct biomass maps at 0.25 ha resolution."
type: Collection
"title: AfriSAR: Aboveground Biomass for Lope, Mabounie, Mondah, and Rabi Sites, Gabon"
"summaries: {'datetime': ['2016-02-01T00:00:00Z'], 'cog_default': None}"
ID: Rabi_AGB_50m
"Bounding Box: [9.859138724101811, -1.9460214484122889, 9.906363400458433, -1.9003139362380275]"
Datetime: 2016-02-01 00:00:00+00:00
boxes: ['-1.94602 9.85914 -1.90031 9.90636']
"links: [{'rel': 'http://esipfed.org/ns/fedsearch/1.1/s3#', 'href': 's3://nasa-maap-data-store/file-staging/nasa-map/AfriSAR_AGB_Maps_1681___1/Rabi_AGB_50m.tif', 'title': 'File to download', 'hreflang': 'en-US'}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#', 'href': 'https://daac.ornl.gov/AFRISAR/guides/AfriSAR_AGB_Maps.html', 'title': 'ORNL DAAC Data Set Documentation (USERS GUIDE)', 'hreflang': 'en-US'}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#', 'href': 'https://dx.doi.org/10.3334/ORNLDAAC/1681', 'title': 'Data set Landing Page DOI URL (DATA SET LANDING PAGE)', 'hreflang': 'en-US'}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#', 'href': 'https:/daac.ornl.gov/daacdata/afrisar/AfriSAR_AGB_Maps/comp/AfriSAR_AGB_Maps.pdf', 'title': 'Data Set Documentation (GENERAL DOCUMENTATION)', 'hreflang': 'en-US'}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#', 'href': 'https:/daac.ornl.gov/daacdata/afrisar/AfriSAR_AGB_Maps/comp/AfriSAR_AGB_Maps_PlotDetails.csv', 'title': 'Data Set Documentation (GENERAL DOCUMENTATION)', 'hreflang': 'en-US'}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'href': 's3://nasa-maap-data-store/file-staging/nasa-map/AfriSAR_AGB_Maps_1681___1', 'hreflang': 'en-US', 'inherited': True}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/documentation#', 'href': 'https://daac.ornl.gov/AFRISAR/guides/AfriSAR_Mondah_Field_Data.html', 'hreflang': 'en-US', 'inherited': True}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/metadata#', 'href': 'https://doi.org/10.3334/ORNLDAAC/1580', 'hreflang': 'en-US', 'inherited': True}, {'rel': 'http://esipfed.org/ns/fedsearch/1.1/documentation#', 'href': 'https://daac.ornl.gov/daacdata/afrisar/AfriSAR_Mondah_Field_Data/comp/AfriSAR_Mondah_Field_Data.pdf', 'hreflang': 'en-US', 'inherited': True}]"


## Use satsearch to discover items in the 'AfriSAR_AGB_Maps_1681' collection

In [44]:
search = cat.search(
    max_items = 15,
    limit = 5,
    collections = collection,
)

## Adding items to an iterable

In [45]:
items = [i.to_dict() for i in search.get_all_items()]

## Extracting s3 link and reading it with rioxarray

In [9]:
item = items[0]
s3_link = item['assets']['data']['href']
da = rioxarray.open_rasterio(s3_link)
da = da.squeeze('band', drop=True)
da

<xarray.DataArray (y: 101, x: 105)>
[10605 values with dtype=float32]
Coordinates:
  * x            (x) float64 5.956e+05 5.956e+05 ... 6.007e+05 6.008e+05
  * y            (y) float64 9.79e+06 9.79e+06 9.79e+06 ... 9.785e+06 9.785e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -9999.0
    scale_factor:   1.0
    add_offset:     0.0

## Visualizing the read data with hvplot

In [21]:
da.hvplot(
    'x', 'y', 
#     frame_height=540,
#     rasterize=True,
#     coastline=True,
#     cmap='viridis'
).redim.range(value=(0,1))

:Image   [x,y]   (value)